In [ ]:
import xgboost
import numpy as np
import pandas as pd 
import pickle

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import r2_score, mean_absolute_error

import matplotlib.pyplot as plt

plt.rcParams['font.family'] = 'DejaVu Serif'
plt.rcParams['font.size'] = 14
plt.rcParams['axes.linewidth'] = 2
plt.style.use('tableau-colorblind10')

import matplotlib.font_manager as fm

font_names = [f.name for f in fm.fontManager.ttflist]

plt.rcParams['font.family'] = 'DejaVu Serif'
plt.rcParams['font.size'] = 14
plt.rcParams['axes.linewidth'] = 2
plt.style.use('tableau-colorblind10')

In [ ]:
def get_figure(figsize=(5.5,4)):

    fig, ax = plt.subplots(figsize=figsize)

    ax.xaxis.set_tick_params(which='major', size=7, width=1.5, direction='in', top='on')
    ax.xaxis.set_tick_params(which='minor', size=4, width=1.5, direction='in', top='on')
    ax.yaxis.set_tick_params(which='major', size=7, width=1.5, direction='in', right='on')
    ax.yaxis.set_tick_params(which='minor', size=4, width=1.5, direction='in', right='on')

    ax.tick_params(bottom=True, top=True, left=True, right=True)
    ax.tick_params(labelbottom=True, labeltop=False, labelleft=True, labelright=False)
    ax.tick_params(direction='in')

    return fig, ax 

## Load Minnesota Data

In [ ]:
import data 

theory_column = 'DeltaGsolv uESE (kcal/mol) - 1'
#'DeltaGsolv uESE (kcal/mol)'
#'DeltaGsolv uESE (kcal/mol) - 0'            
#'DeltaGsolv uESE (kcal/mol) - 1'              
#'DeltaGsolv uESE (kcal/mol) - 2 min'          
#'DeltaGsolv uESE (kcal/mol) - 2 Bolt'         
#'DeltaGsolv uESE (kcal/mol) - 3 min'          
#'DeltaGsolv uESE (kcal/mol) - 3 Bolt'         
#'DeltaGsolv uESE (kcal/mol) - 23 min'         
#'DeltaGsolv uESE (kcal/mol) - 23 Bolt'        
#'DeltaGsolv uESE (kcal/mol)'                  
#'DeltaGsolv SMD (kcal/mol)'

total_df, df_copy = data.load_minnesota_data(theory_column)

nunique_solvents = total_df['Solvent SMILES_solvent'].nunique()
nunique_solutes = total_df['Solute SMILES_solute'].nunique()
print("Number of unique solutes:",nunique_solutes,"\n","Number of unique solvents:",nunique_solvents)

#total_df = total_df.drop(columns=['Solvent SMILES_solvent','Solute SMILES_solute'])

split_df = total_df.copy()
split_y = split_df.pop('Experimental_deltaG')
split_y_calc = split_df.pop('Calculated_deltaG')

print("Number of solute/solvent pairs:",len(split_df))
print("Standard deviation of free energy values:",np.std(split_y_calc.values))

# Load dGsolvDB

In [ ]:
theory_column = 'dGsolv uESE [kcal/mol] - 1'
columns = total_df.columns

dGsolv_df, dGsolv_copy = data.load_dGsolv_data(columns)

nunique_solvents = dGsolv_df['Solvent SMILES_solvent'].nunique()
nunique_solutes = dGsolv_df['Solute SMILES_solute'].nunique()
print("Number of unique solutes:",nunique_solutes,"\n","Number of unique solvents:",nunique_solvents)

dGsolv_df = dGsolv_df.drop(columns=['Solvent SMILES_solvent','Solute SMILES_solute'])

dGsolv_split_df = dGsolv_df.copy()
dGsolv_split_y = dGsolv_split_df.pop('Experimental_deltaG')
dGsolv_split_y_calc = dGsolv_split_df.pop('Calculated_deltaG')

print("Number of solute/solvent pairs:",len(dGsolv_split_df))
print("Standard deviation of free energy values:",np.std(dGsolv_split_y_calc.values))

In [ ]:
total_df = total_df.drop(columns=['Solvent SMILES_solvent','Solute SMILES_solute'])

# Distribution of Data

In [ ]:
from scipy.stats import norm

fig, ax = get_figure()

ax.hist(split_y,bins=50,linewidth=1,edgecolor='black',density=True)
ax.set_ylim(0.0,0.25)
ax.set_xlim(-22,7)
ax.set_xlabel(r'$\Delta G^{exp}_{\mathrm{solv}}$')
ax.set_ylabel(r'P($\Delta G^{exp}_{\mathrm{solv}}$)')

print(np.mean(split_y))
print(np.std(split_y))

mu, std = norm.fit(split_y)

xmin, xmax = plt.xlim()
x = np.linspace(xmin,xmax,100)
p = norm.pdf(x,mu,std)
ax.plot(x,p,'k',linewidth=2)

print("Fit results: mu = %.2f, std = %.2f" %(mu,std))

plt.savefig('./Figures/minnesota_distribution.pdf',bbox_inches='tight')
plt.show()

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

fig, ax = get_figure()

ax.hist(dGsolv_split_y,bins=50,linewidth=1,edgecolor='black',density=True)
ax.set_ylim(0.0,0.25)
ax.set_xlim(-50,7)
ax.set_xlabel(r'$\Delta G^{exp}_{\mathrm{solv}}$')
ax.set_ylabel(r'P($\Delta G^{exp}_{\mathrm{solv}}$)')

print(np.mean(dGsolv_split_y))
print(np.std(dGsolv_split_y))

mu, std = norm.fit(dGsolv_split_y)

xmin, xmax = plt.xlim()
x = np.linspace(xmin,xmax,100)
p = norm.pdf(x,mu,std)
ax.plot(x,p,'k',linewidth=2)

print("Fit results: mu = %.2f, std = %.2f" %(mu,std))

inset_ax = inset_axes(ax, width="150%", height="150%", bbox_to_anchor=(0.175,0.6,.3,.3), bbox_transform=ax.transAxes, loc='upper left')
inset_ax.hist(dGsolv_split_y,bins=50,linewidth=1,edgecolor='black',density=True)
inset_ax.set_xlim(-50,-20)
inset_ax.set_ylim(0.0,0.004)
inset_ax.xaxis.set_tick_params(which='major', size=7, width=1.5, direction='in', top='on')
inset_ax.xaxis.set_tick_params(which='minor', size=4, width=1.5, direction='in', top='on')
inset_ax.yaxis.set_tick_params(which='major', size=7, width=1.5, direction='in', right='on')
inset_ax.yaxis.set_tick_params(which='minor', size=4, width=1.5, direction='in', right='on')

inset_ax.tick_params(bottom=True, top=True, left=True, right=True)
inset_ax.tick_params(labelbottom=True, labeltop=False, labelleft=True, labelright=False)
inset_ax.tick_params(direction='in')


plt.savefig('./Figures/dGSolv_distribution.pdf',bbox_inches='tight')

plt.show()

# Combine Dataset

In [ ]:
import data
combined_df = data.load_combined_data()
#duplicate_rows = combined_df[(combined_df.duplicated(subset=['Solvent SMILES_solvent','Solute SMILES_solute'])==True)].index

combined_split_df = combined_df.copy()
combined_split_y_calc = combined_split_df.pop('Calculated_deltaG')

print("Number of solute/solvent pairs:",len(combined_df))
print("Standard deviation of free energy values:",np.std(combined_split_y_calc.values))
#print("Number of duplicated solute/solvent pairs:", len(duplicate_rows),duplicate_rows.tolist())

# Load XGBoost Models

In [ ]:
with open('./xgboost_hyperopt_5foldCV_quot_uESE23min.pkl','rb') as file:
    model_params = pickle.load(file)

for key in model_params.keys():
    print(key,model_params[key])

# Train Minnesota dataset

## Baseline

In [ ]:
import pickle

skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)

rmse_scores = []
mae_scores = []
r2_scores = []
calc_rmse_scores = []
calc_mae_scores = []
calc_r2_scores = []

for i, (train_index, test_index) in enumerate(skf.split(total_df, df_copy['Stratify_by_rotors'])):
    
    X_train = total_df.iloc[train_index]
    X_test = total_df.iloc[test_index]
    y_train = X_train.pop('Experimental_deltaG')
    y_test = X_test.pop('Experimental_deltaG')

    y_calc = X_train.pop('Calculated_deltaG').values
    y_test_calc = X_test.pop('Calculated_deltaG').values

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_train)

    with open('./xgboost_hyperopt_5foldCV_MLonly.pkl','rb') as file:
        model_params = pickle.load(file)
        print(model_params)
    
    xgb = xgboost.XGBRegressor(**model_params)
    xgb.fit(X_scaled,y_train)

    X_test_scaled = scaler.transform(X_test)

    predictions = xgb.predict(X_test_scaled)
    rmse_scores.append(np.sqrt(np.square(predictions-y_test).mean()))
    mae_scores.append(mean_absolute_error(y_test,predictions))
    r2_scores.append(r2_score(y_test,predictions))
    
    if 'Calculated_deltaG' in X_test.columns:
        calc_mae_scores.append(mean_absolute_error(y_test,X_test['Calculated_deltaG']))
        calc_rmse_scores.append(np.sqrt(np.square(X_test['Calculated_deltaG']-y_test).mean()))
        calc_r2_scores.append(r2_score(y_test,X_test['Calculated_deltaG']))
    else: 
        calc_mae_scores.append(0)
        calc_rmse_scores.append(0)
        calc_r2_scores.append(0)

    fig, ax = get_figure()

    ax.scatter(y_test,predictions,c='red',edgecolors='black')
    ax.plot(np.linspace(min(y_test),max(y_test)),np.linspace(min(y_test),max(y_test)),'--',color='black',linewidth=2)
    ax.set_xlim(-22,5)
    ax.set_ylim(-22,5)
    ax.text(-20,1,r'RMSE = %.2f'%rmse_scores[-1])
    ax.text(-20,-1,r'$R^2$ = %.2f'%r2_scores[-1])
    ax.set_ylabel(r'$\Delta G^{pred}_{solv}$')
    ax.set_xlabel(r'$\Delta G^{true}_{solv}$')
    plt.savefig('./Figures/Parity_Plots/parity_MLonly_minnesota_fivefold_%d_hyperopt.svg'%i,bbox_inches='tight')
    plt.show()

In [ ]:
print("ML Metrics:")
print(np.mean(rmse_scores),np.std(rmse_scores))
print(np.mean(mae_scores),np.std(mae_scores))
print(np.mean(r2_scores),np.std(r2_scores))
print("Calculation metrics:")
print(np.mean(calc_rmse_scores),np.std(calc_rmse_scores))
print(np.mean(calc_mae_scores),np.std(calc_mae_scores))
print(np.mean(calc_r2_scores),np.std(calc_r2_scores))

## Feature-informed

In [ ]:
#Featurization

skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)

rmse_scores = []
mape_scores = []
r2_scores = []
calc_rmse_scores = []
calc_mape_scores = []
calc_r2_scores = []

for i, (train_index, test_index) in enumerate(skf.split(total_df, df_copy['Stratify_by_rotors'])):
    
    X_train = total_df.iloc[train_index]
    X_test = total_df.iloc[test_index]
    y_train = X_train.pop('Experimental_deltaG')
    y_test = X_test.pop('Experimental_deltaG')

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_train)

    with open('./xgboost_hyperopt_5foldCV_informed_uESE1.pkl','rb') as file:
        model_params = pickle.load(file)
        print(model_params)
    xgb = xgboost.XGBRegressor(**model_params)
    xgb.fit(X_scaled,y_train)

    X_test_scaled = scaler.transform(X_test)

    predictions = xgb.predict(X_test_scaled)
    rmse_scores.append(np.sqrt(np.square(predictions-y_test).mean()))
    mape_scores.append(mean_absolute_error(y_test,predictions))
    r2_scores.append(r2_score(y_test,predictions))
    
    if 'Calculated_deltaG' in X_test.columns:
        calc_mape_scores.append(mean_absolute_error(y_test,X_test['Calculated_deltaG']))
        calc_rmse_scores.append(np.sqrt(np.square(X_test['Calculated_deltaG']-y_test).mean()))
        calc_r2_scores.append(r2_score(y_test,X_test['Calculated_deltaG']))
    else: 
        calc_mape_scores.append(0)
        calc_rmse_scores.append(0)
        calc_r2_scores.append(0)

    fig, ax = get_figure()

    ax.scatter(y_test,predictions,c='red',edgecolors='black')
    ax.plot(np.linspace(min(y_test),max(y_test)),np.linspace(min(y_test),max(y_test)),'--',color='black',linewidth=2)
    ax.set_xlim(-22,5)
    ax.set_ylim(-22,5)
    ax.text(-20,1,r'RMSE = %.2f'%rmse_scores[-1])
    ax.text(-20,-1,r'$R^2$ = %.2f'%r2_scores[-1])
    ax.set_ylabel(r'$\Delta G^{pred}_{solv}$')
    ax.set_xlabel(r'$\Delta G^{true}_{solv}$')
    plt.savefig('./Figures/Parity_Plots/parity_informed_minnesota_fivefold_%d_hyperopt.svg'%i,bbox_inches='tight')
    plt.show()

In [ ]:
print("ML Metrics:")
print(np.mean(rmse_scores),np.std(rmse_scores))
print(np.mean(mape_scores),np.std(mape_scores))
print(np.mean(r2_scores),np.std(r2_scores))
print("Calculation metrics:")
print(np.mean(calc_rmse_scores),np.std(calc_rmse_scores))
print(np.mean(calc_mape_scores),np.std(calc_mape_scores))
print(np.mean(calc_r2_scores),np.std(calc_r2_scores))

## Difference

In [ ]:
#Difference 

skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)

rmse_scores = []
mae_scores = []
r2_scores = []
calc_rmse_scores = []
calc_mae_scores = []
calc_r2_scores = []

for i, (train_index, test_index) in enumerate(skf.split(total_df, df_copy['Stratify_by_rotors'])):
    
    X_train = total_df.iloc[train_index]
    X_test = total_df.iloc[test_index]
    y_train_exp = X_train.pop('Experimental_deltaG').values
    y_test_exp = X_test.pop('Experimental_deltaG').values

    y_calc = X_train.pop('Calculated_deltaG').values
    y_test_calc = X_test.pop('Calculated_deltaG').values

    y_train_vals = y_train_exp - y_calc
    y_test_vals = y_test_exp - y_test_calc 
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_train)
    
    with open('./xgboost_hyperopt_5foldCV_diff_uESE23min.pkl','rb') as file:
        model_params = pickle.load(file)
        print(model_params)
    xgb = xgboost.XGBRegressor(**model_params)
    xgb.fit(X_scaled,y_train_vals)

    X_test_scaled = scaler.transform(X_test)

    predictions = xgb.predict(X_test_scaled) + y_test_calc
    rmse_scores.append(np.sqrt(np.square(predictions - y_test_exp).mean()))
    mae_scores.append(mean_absolute_error(y_test_exp,predictions))
    r2_scores.append(r2_score(y_test_exp,predictions))
    
    if 'Calculated_deltaG' in X_test.columns:
        calc_mae_scores.append(mean_absolute_error(y_test_calc,X_test['Calculated_deltaG']))
        calc_rmse_scores.append(np.sqrt(np.square(X_test['Calculated_deltaG']-y_test_calc).mean()))
        calc_r2_scores.append(r2_score(y_test_calc,X_test['Calculated_deltaG']))
    else: 
        calc_mae_scores.append(0)
        calc_rmse_scores.append(0)
        calc_r2_scores.append(0)

    fig, ax = get_figure()

    ax.scatter(y_test_exp,predictions,c='red',edgecolors='black')
    ax.plot(np.linspace(min(y_test_exp),max(y_test_exp)),np.linspace(min(y_test_exp),max(y_test_exp)),'--',color='black',linewidth=2)
    ax.set_xlim(-22,5)
    ax.set_ylim(-22,5)
    ax.text(-20,1,r'RMSE = %.2f'%rmse_scores[-1])
    ax.text(-20,-1,r'$R^2$ = %.2f'%r2_scores[-1])
    ax.set_ylabel(r'$\Delta G^{pred}_{solv}$')
    ax.set_xlabel(r'$\Delta G^{true}_{solv}$')
    plt.savefig('./Figures/Parity_Plots/parity_diff_calc_minnesota_fivefold_%d_uESE23min_hyperopt.svg'%i,bbox_inches='tight')
    plt.show()

In [ ]:
print("ML Metrics:")
print(np.mean(rmse_scores),np.std(rmse_scores))
print(np.mean(mae_scores),np.std(mae_scores))
print(np.mean(r2_scores),np.std(r2_scores))
print("Calculation metrics:")
print(np.mean(calc_rmse_scores),np.std(calc_rmse_scores))
print(np.mean(calc_mae_scores),np.std(calc_mae_scores))
print(np.mean(calc_r2_scores),np.std(calc_r2_scores))

## Ratio

In [ ]:
def obj_quot_fn(y_calc,y_exp):

    def quot_loss(y_true,y_pred) -> np.ndarray:

        gradient = 2.0*y_calc*(y_pred*y_calc - y_exp)
        hessian = 2.0*(y_calc**2) + 0.0*y_calc*(y_pred*y_calc - y_exp)

        return gradient, hessian
    
    return quot_loss

skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)

rmse_scores = []
mape_scores = []
r2_scores = []
calc_rmse_scores = []
calc_mape_scores = []
calc_r2_scores = []

for i, (train_index, test_index) in enumerate(skf.split(total_df, df_copy['Stratify_by_rotors'])):
    
    X_train = total_df.iloc[train_index]
    X_test = total_df.iloc[test_index]
    y_train_exp = X_train.pop('Experimental_deltaG').values
    y_test_exp = X_test.pop('Experimental_deltaG').values

    y_calc = X_train.pop('Calculated_deltaG').values
    y_test_calc = X_test.pop('Calculated_deltaG').values

    y_train_vals = y_train_exp/y_calc
    y_test_vals = y_test_exp/y_test_calc
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_train)

    with open('./xgboost_hyperopt_5foldCV_quot_uESE1.pkl','rb') as file:
        model_params = pickle.load(file)
        print(model_params)
    xgb = xgboost.XGBRegressor(objective=obj_quot_fn(y_calc,y_train_exp),**model_params)
    xgb.fit(X_scaled,y_train_vals)

    X_test_scaled = scaler.transform(X_test)

    predictions = xgb.predict(X_test_scaled)*y_test_calc
    rmse_scores.append(np.sqrt(np.square(predictions - y_test_exp).mean()))
    mape_scores.append(mean_absolute_error(y_test_exp,predictions))
    r2_scores.append(r2_score(y_test_exp,predictions))
    
    if 'Calculated_deltaG' in X_test.columns:
        calc_mape_scores.append(mean_absolute_error(y_test_calc,X_test['Calculated_deltaG']))
        calc_rmse_scores.append(np.sqrt(np.square(X_test['Calculated_deltaG']-y_test_calc).mean()))
        calc_r2_scores.append(r2_score(y_test_calc,X_test['Calculated_deltaG']))
    else: 
        calc_mape_scores.append(0)
        calc_rmse_scores.append(0)
        calc_r2_scores.append(0)

    fig, ax = get_figure()

    ax.scatter(y_test_exp,predictions,c='red',edgecolors='black')
    ax.plot(np.linspace(min(y_test_exp),max(y_test_exp)),np.linspace(min(y_test_exp),max(y_test_exp)),'--',color='black',linewidth=2)
    ax.set_xlim(-22,5)
    ax.set_ylim(-22,5)
    ax.text(-20,1,r'RMSE = %.2f'%rmse_scores[-1])
    ax.text(-20,-1,r'$R^2$ = %.2f'%r2_scores[-1])
    ax.set_ylabel(r'$\Delta G^{pred}_{solv}$')
    ax.set_xlabel(r'$\Delta G^{true}_{solv}$')
    plt.savefig('./Figures/Parity_Plots/parity_quot_calc_minnesota_fivefold_%d_hyperopt.svg'%i,bbox_inches='tight')
    plt.show()


In [ ]:
print("ML Metrics:")
print(np.mean(rmse_scores),np.std(rmse_scores))
print(np.mean(mape_scores),np.std(mape_scores))
print(np.mean(r2_scores),np.std(r2_scores))
print("Calculation metrics:")
print(np.mean(calc_rmse_scores),np.std(calc_rmse_scores))
print(np.mean(calc_mape_scores),np.std(calc_mape_scores))
print(np.mean(calc_r2_scores),np.std(calc_r2_scores))

# Loss vs. Num Data

## Baseline

In [ ]:
#Feature informed
all_rmse_scores = []
all_mae_scores = []
all_r2_scores = []

num_data_list = [20, 50]
percent_data_list = [i/100 for i in range(5,100,5)]
total_data_list = np.concatenate([num_data_list,percent_data_list])

for j in range(1,11):
    rmse_scores = []
    mae_scores = []
    r2_scores = []

    split_df = total_df.copy()
    split_y = split_df.pop('Experimental_deltaG')
    for i in total_data_list:

        if i > 1:
            train_size = int(i)
        else:
            train_size = i
        
        X_train, X_test, y_train, y_test = train_test_split(split_df,split_y,train_size=train_size,stratify=df_copy['Stratify_by_rotors'],random_state=42*j)

        y_calc = X_train.pop('Calculated_deltaG').values
        y_test_calc = X_test.pop('Calculated_deltaG')
        
        #print(len(X_train))
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X_train)

        with open('./xgboost_hyperopt_5foldCV_MLonly.pkl','rb') as file:
            model_params = pickle.load(file)

        xgb = xgboost.XGBRegressor(**model_params)
        xgb.fit(X_scaled,y_train)

        X_test_scaled = scaler.transform(X_test)

        predictions = xgb.predict(X_test_scaled)
        rmse_scores.append(np.sqrt(np.square(predictions-y_test).mean()))
        mae_scores.append(mean_absolute_error(y_test,predictions))
        r2_scores.append(r2_score(y_test,predictions))
        
    all_rmse_scores.append(rmse_scores)
    all_mae_scores.append(mae_scores)
    all_r2_scores.append(r2_scores)


np.savetxt('MLonly_rmse.csv', np.array(all_rmse_scores), delimiter=',',fmt='%.8f')
np.savetxt('MLonly_mae.csv', np.array(all_mae_scores), delimiter=',',fmt='%.8f')
np.savetxt('MLonly_r2.csv', np.array(all_r2_scores), delimiter=',',fmt='%.8f')  

## Feature-informed

In [ ]:
#Feature informed
all_rmse_scores = []
all_mae_scores = []
all_r2_scores = []

all_calc_rmse_scores = []
all_calc_mae_scores = []
all_calc_r2_scores = []

num_data_list = [20, 50]
percent_data_list = [i/100 for i in range(5,100,5)]
total_data_list = np.concatenate([num_data_list,percent_data_list])

for j in range(1,11):
    rmse_scores = []
    mae_scores = []
    r2_scores = []

    calc_rmse_scores = []
    calc_mae_scores = []
    calc_r2_scores = []

    split_df = total_df.copy()
    split_y = split_df.pop('Experimental_deltaG')
    for i in total_data_list:

        if i > 1:
            train_size = int(i)
        else:
            train_size = i
        
        X_train, X_test, y_train, y_test = train_test_split(split_df,split_y,train_size=train_size,stratify=df_copy['Stratify_by_rotors'],random_state=42*j)

        #print(len(X_train))
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X_train)

        with open('./xgboost_hyperopt_5foldCV_informed.pkl','rb') as file:
            model_params = pickle.load(file)

        xgb = xgboost.XGBRegressor(**model_params)
        xgb.fit(X_scaled,y_train)

        X_test_scaled = scaler.transform(X_test)

        predictions = xgb.predict(X_test_scaled)

        rmse_scores.append(np.sqrt(np.square(predictions-y_test).mean()))
        mae_scores.append(mean_absolute_error(y_test,predictions))
        r2_scores.append(r2_score(y_test,predictions))
        
        calc_mae_scores.append(mean_absolute_error(y_test,X_test['Calculated_deltaG']))
        calc_rmse_scores.append(np.sqrt(np.square(X_test['Calculated_deltaG']-y_test).mean()))
        calc_r2_scores.append(r2_score(y_test,X_test['Calculated_deltaG']))

    all_rmse_scores.append(rmse_scores)
    all_mae_scores.append(mae_scores)
    all_r2_scores.append(r2_scores)

    all_calc_rmse_scores.append(calc_rmse_scores)
    all_calc_mae_scores.append(calc_mae_scores)
    all_calc_r2_scores.append(calc_r2_scores)


np.savetxt('informed_calc_rmse.csv', np.array(all_rmse_scores), delimiter=',',fmt='%.8f')
np.savetxt('informed_calc_mae.csv', np.array(all_mae_scores), delimiter=',',fmt='%.8f')
np.savetxt('informed_calc_r2.csv', np.array(all_r2_scores), delimiter=',',fmt='%.8f')      

np.savetxt('calc_only_rmse.csv', np.array(all_calc_rmse_scores), delimiter=',',fmt='%.8f')
np.savetxt('calc_only_mae.csv', np.array(all_calc_mae_scores), delimiter=',',fmt='%.8f')
np.savetxt('calc_only_r2.csv', np.array(all_calc_r2_scores), delimiter=',',fmt='%.8f') 

## Difference

In [ ]:
all_rmse_scores = []
all_mae_scores = []
all_r2_scores = []

num_data_list = [20, 50]
percent_data_list = [i/100 for i in range(5,100,5)]
total_data_list = np.concatenate([num_data_list,percent_data_list])

for j in range(1,11):
    rmse_scores = []
    mae_scores = []
    r2_scores = []

    split_df = total_df.copy()
    split_y = split_df.pop('Experimental_deltaG')
    for i in total_data_list:

        if i > 1:
            train_size = int(i)
        else:
            train_size = i
        
        X_train, X_test, y_train, y_test = train_test_split(split_df,split_y,train_size=train_size,stratify=df_copy['Stratify_by_rotors'],random_state=42*j)
        
        y_train_exp = y_train.values
        y_test_exp = y_test.values

        y_train_calc = X_train.pop('Calculated_deltaG').values
        y_test_calc = X_test.pop('Calculated_deltaG').values

        y_train_vals = y_train_exp - y_train_calc
        y_test_vals = y_test_exp - y_test_calc 
        
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X_train)

        with open('./xgboost_hyperopt_5foldCV_diff.pkl','rb') as file:
            model_params = pickle.load(file)

        xgb = xgboost.XGBRegressor(**model_params)
        xgb.fit(X_scaled,y_train_vals)

        X_test_scaled = scaler.transform(X_test)

        predictions = xgb.predict(X_test_scaled) + y_test_calc
        rmse_scores.append(np.sqrt(np.square(predictions-y_test_exp).mean()))
        mae_scores.append(mean_absolute_error(y_test_exp,predictions))
        r2_scores.append(r2_score(y_test_exp,predictions))
        
    all_rmse_scores.append(rmse_scores)
    all_mae_scores.append(mae_scores)
    all_r2_scores.append(r2_scores)


np.savetxt('diff_calc_rmse.csv', np.array(all_rmse_scores), delimiter=',',fmt='%.8f')
np.savetxt('diff_calc_mae.csv', np.array(all_mae_scores), delimiter=',',fmt='%.8f')
np.savetxt('diff_calc_r2.csv', np.array(all_r2_scores), delimiter=',',fmt='%.8f')


## Ratio

In [ ]:
all_rmse_scores = []
all_mae_scores = []
all_r2_scores = []

num_data_list = [20, 50]
percent_data_list = [i/100 for i in range(5,100,5)]
total_data_list = np.concatenate([num_data_list,percent_data_list])

for j in range(1,11):
    rmse_scores = []
    mae_scores = []
    r2_scores = []

    split_df = total_df.copy()
    split_y = split_df.pop('Experimental_deltaG')
    for i in total_data_list:

        if i > 1:
            train_size = int(i)
        else:
            train_size = i
        
        X_train, X_test, y_train, y_test = train_test_split(split_df,split_y,train_size=train_size,stratify=df_copy['Stratify_by_rotors'],random_state=42*j)
        
        y_train_exp = y_train.values
        y_test_exp = y_test.values

        y_train_calc = X_train.pop('Calculated_deltaG').values
        y_test_calc = X_test.pop('Calculated_deltaG').values

        y_train_vals = y_train_exp/y_train_calc
        y_test_vals = y_test_exp/y_test_calc 
        
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X_train)

        with open('./xgboost_hyperopt_5foldCV_quot.pkl','rb') as file:
            model_params = pickle.load(file)

        xgb = xgboost.XGBRegressor(objective=obj_quot_fn(y_train_calc,y_train_exp),**model_params)
        xgb.fit(X_scaled,y_train_vals)

        X_test_scaled = scaler.transform(X_test)

        predictions = xgb.predict(X_test_scaled) * y_test_calc
        rmse_scores.append(np.sqrt(np.square(predictions-y_test_exp).mean()))
        mae_scores.append(mean_absolute_error(y_test_exp,predictions))
        r2_scores.append(r2_score(y_test_exp,predictions))
        
    all_rmse_scores.append(rmse_scores)
    all_mae_scores.append(mae_scores)
    all_r2_scores.append(r2_scores)


np.savetxt('quot_calc_rmse.csv', np.array(all_rmse_scores), delimiter=',',fmt='%.8f')
np.savetxt('quot_calc_mae.csv', np.array(all_mae_scores), delimiter=',',fmt='%.8f')
np.savetxt('quot_calc_r2.csv', np.array(all_r2_scores), delimiter=',',fmt='%.8f')


# Performance on dGsolvDB

## uESE-1 (Extrapolate & Interpolate)

In [ ]:
split_df = total_df.copy()
split_y = split_df.pop('Experimental_deltaG').values

split_y_calc = split_df.pop('Calculated_deltaG').values

X_test = dGsolv_df.copy()
y_test_dGsolv = X_test.pop('Experimental_deltaG').values

y_test_calc = X_test.pop('Calculated_deltaG').values

rmse = np.sqrt(np.square(y_test_calc-y_test_dGsolv).mean())
mae = mean_absolute_error(y_test_dGsolv,y_test_calc)
r2 = r2_score(y_test_dGsolv,y_test_calc)

fig, ax = get_figure()

ax.scatter(y_test_dGsolv,y_test_calc,color='red',edgecolors='black')
ax.plot(np.linspace(min(y_test_dGsolv),max(y_test_dGsolv)),np.linspace(min(y_test_dGsolv),max(y_test_dGsolv)),'--',color='black',linewidth=2)
ax.plot(np.linspace(-20.3,-20.3),np.linspace(-60,10),'--',color='gray',linewidth=1)
ax.text(-18,-40,r"RMSE = %.2f""\n"r"$R^2$ = %.2f"%(rmse,r2))
ax.set_ylim(-50,10)
ax.set_xlim(-50,10)
ax.set_ylabel(r'$\mathbf{\Delta G^{calc}_{solv}}$')
ax.set_xlabel(r'$\mathbf{\Delta G^{exp}_{solv}}$')
plt.savefig("./Figures/Parity_Plots/parity_calconly_dGsolv_extrapolate.pdf",bbox_inches='tight')
plt.show()

## uESE-1 (Interpolate)

In [ ]:
split_df = total_df.copy()
split_y = split_df.pop('Experimental_deltaG').values

split_y_calc = split_df.pop('Calculated_deltaG').values

X_test = dGsolv_df.copy()
X_test = X_test[(X_test['Experimental_deltaG']>=np.min(split_y)) & (X_test['Experimental_deltaG']<=np.max(split_y))]

y_test_dGsolv = X_test.pop('Experimental_deltaG').values

y_test_calc = X_test.pop('Calculated_deltaG').values

rmse = np.sqrt(np.square(y_test_calc-y_test_dGsolv).mean())
mae = mean_absolute_error(y_test_dGsolv,y_test_calc)
r2 = r2_score(y_test_dGsolv,y_test_calc)

fig, ax = get_figure()

ax.scatter(y_test_dGsolv,y_test_calc,color='red',edgecolors='black')
ax.plot(np.linspace(min(y_test_dGsolv)-5,max(y_test_dGsolv)+5),np.linspace(min(y_test_dGsolv)-5,max(y_test_dGsolv)+5),'--',color='black',linewidth=2)
#ax.plot(np.linspace(-20.3,-20.3),np.linspace(-60,10),'--',color='gray',linewidth=1)
ax.text(-5,-21,r"RMSE = %.2f""\n"r"$R^2$ = %.2f"%(rmse,r2))
ax.set_ylim(-25,7)
ax.set_xlim(-25,7)
ax.set_ylabel(r'$\mathbf{\Delta G^{calc}_{solv}}$')
ax.set_xlabel(r'$\mathbf{\Delta G^{exp}_{solv}}$')
plt.savefig("./Figures/Parity_Plots/parity_calconly_dGsolv_interpolate.pdf",bbox_inches='tight')
plt.show()

## Baseline (Extrapolate & Interpolate)

In [ ]:
split_df = total_df.copy()
split_y = split_df.pop('Experimental_deltaG').values

print(np.min(split_y))

split_y_calc = split_df.pop('Calculated_deltaG').values
        
scaler = StandardScaler()
X_scaled = scaler.fit_transform(split_df)

with open('./xgboost_hyperopt_5foldCV_MLonly.pkl','rb') as file:
    model_params = pickle.load(file)

xgb = xgboost.XGBRegressor(**model_params)
xgb.fit(X_scaled,split_y)

X_test = dGsolv_df.copy()
y_test_dGsolv = X_test.pop('Experimental_deltaG').values

y_test_calc = X_test.pop('Calculated_deltaG').values

X_test_scaled = scaler.transform(X_test)

predictions = xgb.predict(X_test_scaled)
rmse = np.sqrt(np.square(predictions-y_test_dGsolv).mean())
mae = mean_absolute_error(y_test_dGsolv,predictions)
r2 = r2_score(y_test_dGsolv,predictions)

fig, ax = get_figure()

ax.scatter(y_test_dGsolv,predictions,color='red',edgecolors='black')
ax.plot(np.linspace(min(y_test_dGsolv),max(y_test_dGsolv)),np.linspace(min(y_test_dGsolv),max(y_test_dGsolv)),'--',color='black',linewidth=2)
ax.plot(np.linspace(-20.3,-20.3),np.linspace(-60,10),'--',color='gray',linewidth=1)
ax.text(-18,-40,r"RMSE = %.2f""\n"r"$R^2$ = %.2f"%(rmse,r2))
ax.set_ylim(-50,10)
ax.set_xlim(-50,10)
ax.set_ylabel(r'$\mathbf{\Delta G^{pred}_{solv}}$')
ax.set_xlabel(r'$\mathbf{\Delta G^{exp}_{solv}}$')
plt.savefig("./Figures/Parity_Plots/parity_MLonly_dGsolv_extrapolate.pdf",bbox_inches='tight')
plt.show()

## Baseline (Interpolate)

In [ ]:
split_df = total_df.copy()
split_y = split_df.pop('Experimental_deltaG').values

split_y_calc = split_df.pop('Calculated_deltaG').values
        
scaler = StandardScaler()
X_scaled = scaler.fit_transform(split_df)

with open('./xgboost_hyperopt_5foldCV_MLonly.pkl','rb') as file:
    model_params = pickle.load(file)

xgb = xgboost.XGBRegressor(**model_params)
xgb.fit(X_scaled,split_y)

X_test = dGsolv_df.copy()
X_test = X_test[(X_test['Experimental_deltaG']>=np.min(split_y)) & (X_test['Experimental_deltaG']<=np.max(split_y))]
y_test_dGsolv = X_test.pop('Experimental_deltaG').values

y_test_calc = X_test.pop('Calculated_deltaG').values

X_test_scaled = scaler.transform(X_test)

predictions = xgb.predict(X_test_scaled)
rmse = np.sqrt(np.square(predictions-y_test_dGsolv).mean())
mae = mean_absolute_error(y_test_dGsolv,predictions)
r2 = r2_score(y_test_dGsolv,predictions)

fig, ax = get_figure()

ax.scatter(y_test_dGsolv,y_test_calc,color='red',edgecolors='black')
ax.plot(np.linspace(min(y_test_dGsolv)-5,max(y_test_dGsolv)+5),np.linspace(min(y_test_dGsolv)-5,max(y_test_dGsolv)+5),'--',color='black',linewidth=2)
#ax.plot(np.linspace(-20.3,-20.3),np.linspace(-60,10),'--',color='gray',linewidth=1)
ax.text(-5,-21,r"RMSE = %.2f""\n"r"$R^2$ = %.2f"%(rmse,r2))
ax.set_ylim(-25,7)
ax.set_xlim(-25,7)
ax.set_ylabel(r'$\mathbf{\Delta G^{pred}_{solv}}$')
ax.set_xlabel(r'$\mathbf{\Delta G^{exp}_{solv}}$')
plt.savefig("./Figures/Parity_Plots/parity_MLonly_dGsolv_interpolate.pdf",bbox_inches='tight')
plt.show()

## Feature-informed (Extrapolate & Interpolate)

In [ ]:
split_df = total_df.copy()
split_y = split_df.pop('Experimental_deltaG')
        
scaler = StandardScaler()
X_scaled = scaler.fit_transform(split_df)

with open('./xgboost_hyperopt_5foldCV_informed_uESE1.pkl','rb') as file:
    model_params = pickle.load(file)

xgb = xgboost.XGBRegressor(**model_params)
xgb.fit(X_scaled,split_y)

X_test = dGsolv_df.copy()
y_test_dGsolv = X_test.pop('Experimental_deltaG')

X_test_scaled = scaler.transform(X_test)

predictions = xgb.predict(X_test_scaled)
rmse = np.sqrt(np.square(predictions-y_test_dGsolv).mean())
mae = mean_absolute_error(y_test_dGsolv,predictions)
r2 = r2_score(y_test_dGsolv,predictions)

fig, ax = get_figure()

ax.scatter(y_test_dGsolv,predictions,color='red',edgecolors='black')
ax.plot(np.linspace(min(y_test_dGsolv),max(y_test_dGsolv)),np.linspace(min(y_test_dGsolv),max(y_test_dGsolv)),'--',color='black',linewidth=2)
ax.plot(np.linspace(-20.3,-20.3),np.linspace(-60,10),'--',color='gray',linewidth=1)
ax.text(-18,-40,r"RMSE = %.2f""\n"r"$R^2$ = %.2f"%(rmse,r2))
ax.set_ylim(-50,10)
ax.set_xlim(-50,10)
ax.set_ylabel(r'$\mathbf{\Delta G^{pred}_{solv}}$')
ax.set_xlabel(r'$\mathbf{\Delta G^{exp}_{solv}}$')
plt.savefig("./Figures/Parity_Plots/parity_informed_dGsolv_extrapolate.pdf",bbox_inches='tight')
plt.show()

## Feature-informed (Interpolate)

In [ ]:
split_df = total_df.copy()
split_y = split_df.pop('Experimental_deltaG')
        
scaler = StandardScaler()
X_scaled = scaler.fit_transform(split_df)

with open('./xgboost_hyperopt_5foldCV_informed_uESE1.pkl','rb') as file:
    model_params = pickle.load(file)

xgb = xgboost.XGBRegressor(**model_params)
xgb.fit(X_scaled,split_y)

X_test = dGsolv_df.copy()
X_test = X_test[(X_test['Experimental_deltaG']>=np.min(split_y)) & (X_test['Experimental_deltaG']<=np.max(split_y))]
y_test_dGsolv = X_test.pop('Experimental_deltaG')

X_test_scaled = scaler.transform(X_test)

predictions = xgb.predict(X_test_scaled)
rmse = np.sqrt(np.square(predictions-y_test_dGsolv).mean())
mae = mean_absolute_error(y_test_dGsolv,predictions)
r2 = r2_score(y_test_dGsolv,predictions)

fig, ax = get_figure()

ax.scatter(y_test_dGsolv,y_test_calc,color='red',edgecolors='black')
ax.plot(np.linspace(min(y_test_dGsolv)-5,max(y_test_dGsolv)+5),np.linspace(min(y_test_dGsolv)-5,max(y_test_dGsolv)+5),'--',color='black',linewidth=2)
#ax.plot(np.linspace(-20.3,-20.3),np.linspace(-60,10),'--',color='gray',linewidth=1)
ax.text(-5,-21,r"RMSE = %.2f""\n"r"$R^2$ = %.2f"%(rmse,r2))
ax.set_ylim(-25,7)
ax.set_xlim(-25,7)
ax.set_ylabel(r'$\mathbf{\Delta G^{pred}_{solv}}$')
ax.set_xlabel(r'$\mathbf{\Delta G^{exp}_{solv}}$')
plt.savefig("./Figures/Parity_Plots/parity_informed_dGsolv_interpolate.pdf",bbox_inches='tight')
plt.show()

## Difference (Extrapolation & Interpolation)

In [ ]:
split_df = total_df.copy()
split_y = split_df.pop('Experimental_deltaG').values

y_calc = split_df.pop('Calculated_deltaG').values
y_train = split_y - y_calc
        
scaler = StandardScaler()
X_scaled = scaler.fit_transform(split_df)

with open('./xgboost_hyperopt_5foldCV_diff_uESE1.pkl','rb') as file:
    model_params = pickle.load(file)

xgb = xgboost.XGBRegressor(**model_params)
xgb.fit(X_scaled,y_train)

X_test = dGsolv_df.copy()
y_test_dGsolv = X_test.pop('Experimental_deltaG').values
y_test_calc = X_test.pop('Calculated_deltaG').values

X_test_scaled = scaler.transform(X_test)

predictions = xgb.predict(X_test_scaled) + y_test_calc
rmse = np.sqrt(np.square(predictions-y_test_dGsolv).mean())
mae = mean_absolute_error(y_test_dGsolv,predictions)
r2 = r2_score(y_test_dGsolv,predictions)

fig, ax = get_figure()

ax.scatter(y_test_dGsolv,predictions,color='red',edgecolors='black')
ax.plot(np.linspace(min(y_test_dGsolv),max(y_test_dGsolv)),np.linspace(min(y_test_dGsolv),max(y_test_dGsolv)),'--',color='black',linewidth=2)
ax.plot(np.linspace(-20.3,-20.3),np.linspace(-60,10),'--',color='gray',linewidth=1)
ax.text(-18,-40,r"RMSE = %.2f""\n"r"$R^2$ = %.2f"%(rmse,r2))
ax.set_ylim(-52,10)
ax.set_xlim(-52,10)
ax.set_ylabel(r'$\mathbf{\Delta G^{pred}_{solv}}$')
ax.set_xlabel(r'$\mathbf{\Delta G^{exp}_{solv}}$')
plt.savefig("./Figures/Parity_Plots/parity_diff_calc_dGsolv_extrapolate.pdf",bbox_inches='tight')
plt.show()

## Difference (Interpolate)

In [ ]:
split_df = total_df.copy()
split_y = split_df.pop('Experimental_deltaG').values

y_calc = split_df.pop('Calculated_deltaG').values
y_train = split_y - y_calc
        
scaler = StandardScaler()
X_scaled = scaler.fit_transform(split_df)

with open('./xgboost_hyperopt_5foldCV_diff_uESE1.pkl','rb') as file:
    model_params = pickle.load(file)

xgb = xgboost.XGBRegressor(**model_params)
xgb.fit(X_scaled,y_train)

X_test = dGsolv_df.copy()
X_test = X_test[(X_test['Experimental_deltaG']>=np.min(split_y)) & (X_test['Experimental_deltaG']<=np.max(split_y))]
y_test_dGsolv = X_test.pop('Experimental_deltaG').values
y_test_calc = X_test.pop('Calculated_deltaG').values

X_test_scaled = scaler.transform(X_test)

predictions = xgb.predict(X_test_scaled) + y_test_calc
rmse = np.sqrt(np.square(predictions-y_test_dGsolv).mean())
mae = mean_absolute_error(y_test_dGsolv,predictions)
r2 = r2_score(y_test_dGsolv,predictions)

fig, ax = get_figure()

ax.scatter(y_test_dGsolv,y_test_calc,color='red',edgecolors='black')
ax.plot(np.linspace(min(y_test_dGsolv)-5,max(y_test_dGsolv)+5),np.linspace(min(y_test_dGsolv)-5,max(y_test_dGsolv)+5),'--',color='black',linewidth=2)
#ax.plot(np.linspace(-20.3,-20.3),np.linspace(-60,10),'--',color='gray',linewidth=1)
ax.text(-5,-21,r"RMSE = %.2f""\n"r"$R^2$ = %.2f"%(rmse,r2))
ax.set_ylim(-25,7)
ax.set_xlim(-25,7)
ax.set_ylabel(r'$\mathbf{\Delta G^{pred}_{solv}}$')
ax.set_xlabel(r'$\mathbf{\Delta G^{exp}_{solv}}$')
plt.savefig("./Figures/Parity_Plots/parity_diff_calc_dGsolv_interpolate.pdf",bbox_inches='tight')
plt.show()

## Ratio (Extrapolate & Interpolate)

In [ ]:
def obj_quot_fn(y_calc,y_exp):

    def quot_loss(y_true,y_pred) -> np.ndarray:

        gradient = 2.0*y_calc*(y_pred*y_calc - y_exp)
        hessian = 2.0*(y_calc**2) + 0.0*y_calc*(y_pred*y_calc - y_exp)

        return gradient, hessian
    
    return quot_loss

split_df = total_df.copy()
split_y = split_df.pop('Experimental_deltaG').values

y_calc = split_df.pop('Calculated_deltaG').values
y_train = split_y/y_calc
        
scaler = StandardScaler()
X_scaled = scaler.fit_transform(split_df)

with open('./xgboost_hyperopt_5foldCV_quot_uESE1.pkl','rb') as file:
    model_params = pickle.load(file)
xgb = xgboost.XGBRegressor(objective=obj_quot_fn(y_calc,split_y),**model_params)
xgb.fit(X_scaled,y_train)

X_test = dGsolv_df.copy()
y_test_dGsolv = X_test.pop('Experimental_deltaG').values
y_test_calc = X_test.pop('Calculated_deltaG').values

X_test_scaled = scaler.transform(X_test)

predictions = xgb.predict(X_test_scaled) * y_test_calc
rmse = np.sqrt(np.square(predictions-y_test_dGsolv).mean())
mae = mean_absolute_error(y_test_dGsolv,predictions)
r2 = r2_score(y_test_dGsolv,predictions)

fig, ax = get_figure()

ax.scatter(y_test_dGsolv,predictions,color='red',edgecolors='black')
ax.plot(np.linspace(min(y_test_dGsolv),max(y_test_dGsolv)),np.linspace(min(y_test_dGsolv),max(y_test_dGsolv)),'--',color='black',linewidth=2)
ax.plot(np.linspace(-20.3,-20.3),np.linspace(-60,10),'--',color='gray',linewidth=1)
ax.text(-18,-40,r"RMSE = %.2f""\n"r"$R^2$ = %.2f"%(rmse,r2))
ax.set_ylim(-50,10)
ax.set_xlim(-50,10)
ax.set_ylabel(r'$\mathbf{\Delta G^{pred}_{solv}}$')
ax.set_xlabel(r'$\mathbf{\Delta G^{exp}_{solv}}$')
plt.savefig("./Figures/Parity_Plots/parity_quot_calc_dGsolv_extrapolate.pdf",bbox_inches='tight')
plt.show()

## Ratio (Interpolate)

In [ ]:
def obj_quot_fn(y_calc,y_exp):

    def quot_loss(y_true,y_pred) -> np.ndarray:

        gradient = 2.0*y_calc*(y_pred*y_calc - y_exp)
        hessian = 2.0*(y_calc**2) + 0.0*y_calc*(y_pred*y_calc - y_exp)

        return gradient, hessian
    
    return quot_loss

split_df = total_df.copy()
split_y = split_df.pop('Experimental_deltaG').values

y_calc = split_df.pop('Calculated_deltaG').values
y_train = split_y/y_calc
        
scaler = StandardScaler()
X_scaled = scaler.fit_transform(split_df)

with open('./xgboost_hyperopt_5foldCV_quot_uESE1.pkl','rb') as file:
    model_params = pickle.load(file)
xgb = xgboost.XGBRegressor(objective=obj_quot_fn(y_calc,split_y),**model_params)
xgb.fit(X_scaled,y_train)

X_test = dGsolv_df.copy()
X_test = X_test[(X_test['Experimental_deltaG']>=np.min(split_y)) & (X_test['Experimental_deltaG']<=np.max(split_y))]
y_test_dGsolv = X_test.pop('Experimental_deltaG').values
y_test_calc = X_test.pop('Calculated_deltaG').values

X_test_scaled = scaler.transform(X_test)

predictions = xgb.predict(X_test_scaled) * y_test_calc
rmse = np.sqrt(np.square(predictions-y_test_dGsolv).mean())
mae = mean_absolute_error(y_test_dGsolv,predictions)
r2 = r2_score(y_test_dGsolv,predictions)

fig, ax = get_figure()

ax.scatter(y_test_dGsolv,y_test_calc,color='red',edgecolors='black')
ax.plot(np.linspace(min(y_test_dGsolv)-5,max(y_test_dGsolv)+5),np.linspace(min(y_test_dGsolv)-5,max(y_test_dGsolv)+5),'--',color='black',linewidth=2)
#ax.plot(np.linspace(-20.3,-20.3),np.linspace(-60,10),'--',color='gray',linewidth=1)
ax.text(-5,-21,r"RMSE = %.2f""\n"r"$R^2$ = %.2f"%(rmse,r2))
ax.set_ylim(-25,7)
ax.set_xlim(-25,7)
ax.set_ylabel(r'$\mathbf{\Delta G^{pred}_{solv}}$')
ax.set_xlabel(r'$\mathbf{\Delta G^{exp}_{solv}}$')
plt.savefig("./Figures/Parity_Plots/parity_quot_calc_dGsolv_interpolate.pdf",bbox_inches='tight')
plt.show()

# Active Learning

In [ ]:
total_df = combined_df.copy()
data = np.loadtxt('./active_learning_informed_MaxDiff_v2_cleaned.txt',usecols=[9,10,11,12,13],dtype=int)

In [ ]:
all_rmse_scores = []
all_mae_scores = []
all_r2_scores = []

index_val = 0
all_min_deltaG = []

for j in range(1,11):
    rmse_scores = []
    mae_scores = []
    r2_scores = []

    split_df = total_df.copy()
    split_y = split_df.pop('Experimental_deltaG')

    X_train, X_test, y_train, y_test = train_test_split(split_df,split_y,train_size=20,random_state=42*j)

    X_train = X_train.reset_index(drop=True) #need to reset so that test set can be selected by list of index values
    X_test = X_test.reset_index(drop=True) #need to reset so that test set can be selected by list of index values

    y_train_calc = X_train.pop('Calculated_deltaG')
    y_test_calc = X_test.pop('Calculated_deltaG')

    next_sample = None
    min_deltaG = []
    for i in range(0,201):

        if next_sample is not None:
            X_train = pd.concat([X_train,X_test.iloc[next_sample]],ignore_index=True).reset_index(drop=True)
            y_train = pd.concat([y_train,y_test.iloc[next_sample]],ignore_index=True).reset_index(drop=True)
            
            X_test = X_test.drop(X_test.index[next_sample]).reset_index(drop=True)
            y_test = y_test.drop(y_test.index[next_sample]).reset_index(drop=True)

            y_train_calc = pd.concat([y_train_calc,y_test_calc.iloc[next_sample]],ignore_index=True).reset_index(drop=True)              
            y_test_calc = y_test_calc.drop(y_test_calc.index[next_sample]).reset_index(drop=True)

        min_deltaG.append(np.min(y_train_calc))
        next_sample = data[index_val]
        index_val+=1 

    all_min_deltaG.append(min_deltaG)

In [ ]:
cmap = plt.get_cmap('viridis', 10)
colors = [cmap(i) for i in range(0,10)]

fig, ax = get_figure()


for j in range(0,10):
    ax.plot(np.arange(0,201),all_min_deltaG[j],color=colors[j])
    # print(np.argmin(all_min_deltaG[j]),all_min_deltaG[j][np.argmin(all_min_deltaG[j])])

# ax.set_xlim(125,201)
ax.set_xlabel('Iteration',fontweight='bold')
ax.set_ylabel(r'$\mathbf{\Delta G^{exp}_{solv}}$')
plt.savefig('./Figures/GPR_informed_MaxDiff_minloc.pdf',bbox_inches='tight')
plt.show()

# Feature Importance (SHAP)

In [ ]:
import shap 

skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)

with open('./xgboost_hyperopt_5foldCV_informed_uESE23min.pkl','rb') as file:
    model_params = pickle.load(file)

for i, (train_index, test_index) in enumerate(skf.split(total_df, df_copy['Stratify_by_rotors'])):
    
    fig, ax = get_figure()

    X_train = total_df.iloc[train_index]
    X_test = total_df.iloc[test_index]
    y_train = X_train.pop('Experimental_deltaG')
    y_test = X_test.pop('Experimental_deltaG')

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_train)

    xgb = xgboost.XGBRegressor(**model_params)
    xgb.fit(X_scaled,y_train)

    X_test_scaled = scaler.transform(X_test)

    explainer = shap.Explainer(xgb)
    shap_values = explainer(X_test)
    # shap.plots.beeswarm(shap_values)
    shap.plots.bar(shap_values,max_display=5,show=False,ax=ax)
    plt.savefig('./Figures/shap_informed_uESE23min_fold_%s.pdf'%str(i),bbox_inches='tight')